[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

## Enunciado del  problema.

Se desea obtener el nombre de los asistentes a una sesión de Gotomeeting mediante las bitácora de chat de dicha plataforma.

* Las bitácoras son guardadas automáticamente en archivos con extensión *rtf* al terminar una sesión. 
* Los nombres de los asistentes siempre se ponen antes de cada comentario, indicando a quién va dirigido el mensaje mediante paréntesis.
* Los caracteres que no se encuentran el el código ASCII son sustituidos mediante un caracter de escape " *\\'* " seguido por el número hexadecimal correspondiente a dicho caracter en la codificación [*Latin-1*](https://es.wikipedia.org/wiki/ISO/IEC_8859-1).
* Los asistentes pueden escribir su nombre en mayúsculas y minúsculas y es común que no coloquen los acentos de la misma forma en cada sesión.

```
\cf3\b Joruiz Kartate (a Todos)\b0 : \cf1 03:09 p. m.: Buenas tardes, entendido\cf2\par
\cf3\b Lop\'den Rasquam (a Todos)\b0 : \cf1 03:09 p. m.: y buenas tardes \cf2\par
```

## Código explicado.

### La función *escape()*.

Esta función realiza la conversión de los caracteres escapados en codificación *Latin-1*.

In [ ]:
def escape(cadena):
    while "\\'" in cadena:
        indice = cadena.index("\\'")
        original = cadena[indice: indice + 4] 
        cadena = cadena.replace(original, 
                                bytes.fromhex(original[-2:]).decode('latin1'))
    return cadena

**Ejemplo:**

In [ ]:
escape("Esta es la í con acento: \\'ed")

### La función *abre_archivo()*.

Esta función abre un archivo de texto y regresa in objeto de tipo *set*  y extrae de cada línea la cadena de caracteres que se ajusta a los siguientes criterios. 
* Selecciona la línea del archivo en las que existen los caracteres "*("* y los caracteres "*)\\b*".
* Aplica la función *escape()* a cada línea.
* Recorta el texto hasta antes de encontrar el signo de apertura de paréntesis precedido por un espacio"* (*".
* Desecha los primeros 7 caracteres de cada línea.
* Sustituye caracteres especiales.
* Transforma el texto en minúsculas.
* Crea un elemento set con cada línea procesada.

In [ ]:
def abre_archivo(ruta):
    mapa = str.maketrans('áéíóúüàèìòùñ', 'aeiouuaeioun')
    with open(ruta, 'r') as archivo:
        return set((escape(linea.partition(' (')[0][6:]).strip().lower().translate(mapa)\
                    for linea in archivo.readlines() if '(' in linea and ')\\b' in linea))

**Ejemplo:**

El archivo [data/Registro de conversaciones 01_15 16_00.rtf](data/Registro%20de%20conversaciones%2001_15%2016_00.rtf)

In [ ]:
abre_archivo("data/Registro de conversaciones 01_15 16_00.rtf")

### La función *asistentes()*.

In [ ]:
def asistentes(lista):
    conjunto = set()
    for elemento in lista:
        conjunto.update(elemento)
    return conjunto

**Ejemplo:**

In [ ]:
asistencia = ({'Luis', 'Juan', 'Jorge'},
          {'Arturo', 'Juan', 'Ivan', 'Luis'},
          {'Guadalupe', 'Marco', 'Luis'})

In [ ]:
asistentes(asistencia)

## La función *lista_sesiones*. 

Esta función va a crear una lista de asistencia para todas las sesiones.

In [ ]:
def lista_sesiones(sesiones):
    alumnos = asistentes(sesiones)
    control = {}
    for sesion in sesiones:
        for alumno in alumnos:
            asistio = 0
            if alumno in sesion:
                asistio = 1
            if alumno in control:
                control[alumno].append(asistio)
                control[alumno][0] += asistio
            else:
                control[alumno] = [asistio, asistio]
    return control

**Ejemplo:*

In [ ]:
lista_sesiones(asistencia)

### La función *crea_csv*.

#### Importación de los módulos requeridos.

In [ ]:
from os import listdir
import csv

Esta función crea el objeto *conjunto* el cual contiene la relación de asistencias de todas las sesiones.

In [ ]:
def crea_csv(archivos, ruta='relacion.csv'):
    conjunto = lista_sesiones([abre_archivo(archivo) for archivo in archivos])
    with open(ruta, 'wt') as archivo:
        relacion = csv.writer(archivo)
        for alumno in conjunto:
            fila = [alumno]
            fila.extend(conjunto[alumno])
            relacion.writerow(fila)

### La función *main()*.

Esta es la función que ejecuta el proceso. 
* Realizará y ordenará el listado de todos los archivos del subdirectorio *data*  con la extensión definida en *extension* en la ruta definida en *directorio*.   

In [ ]:
def main(directorio='./', extension='.rtf'):
    directorio = [directorio + item for item in listdir('data') if item.endswith('.rtf')]
    directorio.sort()
    crea_csv(directorio)

In [ ]:
if __name__ == '__main__':
    main('data/')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>